In [1]:
!ls

drive  sample_data


In [31]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os

In [ ]:
#Lendo modelo
model = torch.hub.load('/content/drive/MyDrive/yolov5_LIV/yolov5', 'custom', path='/content/drive/MyDrive/yolov5_LIV/yolov5/runs/train/exp59/weights/best.pt', source='local') 

#Modelo irá mostrar apenas predições com 70% de certeza
model.conf = 0.7

In [36]:
#Setando o caminho onde o vídeo está localizado
path_video = '/content/drive/MyDrive/video/v1.mp4'

#Caminho onde os arquivos serão salvos
save_path = '/content/drive/MyDrive/imagens_auto_anotadas/'

try:
  os.mkdir( '/content/drive/MyDrive/imagens_auto_anotadas/')
  print('Criando o diretório')
except:
  print('Diretório encontrado')

#Carregando o vídeo 
cap = cv2.VideoCapture(path_video)

#Variáveis para se utilizar no salvamento dos frames e anotações
num_frame = 0
name_frame = path_vid.rsplit('/',1)[1].rsplit('.',1)[0]+'_frame_'


#Rodando o loop equanto o vídeo não estiver finalizado 
while True:
  
  #Lendo cada Frame
  ret, frame = cap.read()
  
  #Variáveis para salvamento
  num_frame +=1 
  save_name = name_frame+str(num_frame)
  
  if ret == True:
      #Converten do tipo numpy array para PIL.image (aumenta a precisão por algum motivo)
      frame_pil = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame_pil = Image.fromarray(frame)

      #Enviando a imagem para a rede
      results = model(frame_pil,  size=1280)
      
      #Transformando as predições do modelo em formato pandas DataFrame 
      dados = results.pandas().xyxy[0]
      
      #Realizando o préprocessamento necessário para transformar as predições no 
      #formato padrão das anotações do modelo YOLO 
      im_w, im_h = im.size
      dados = dados[['class','xmin', 'ymin', 'xmax', 'ymax']]
      dados['W'] = dados['xmax']-dados['xmin']
      dados['H'] = dados['ymax']-dados['ymin']
      dados['Norm(xMin)'] = (dados['xmin']+(dados['W']/2))/im_w
      dados['Norm(yMin)'] = (dados['ymin']+(dados['H']/2))/im_h
      dados['Norm(W)'] = (dados['W'])/im_w
      dados['Norm(H)'] = (dados['H'])/im_h
      dados = dados.drop(['xmin',	'ymin',	'xmax',	'ymax', 'H','W'], axis = 1)

      #Transformando o Dataframe em txt
      textfile = open(save_path+save_name+".txt", "w")
      for i in range(len(dados)):
          lista = dados.iloc[i].to_list()
          lista[0] = int(lista[0])
          listToStr = ' '.join(map(str, lista))
          textfile.write(listToStr + "\n")

      textfile.close()
      cv2.imwrite(save_path+save_name+".jpg", frame)



Criando o diretório
Diretório encontrado


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: ignored

In [5]:
im = Image.open('/content/drive/MyDrive/video/v1.mp4')

In [6]:
results = model(im,  size=1280)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
dados = results.pandas().xyxy[0]

In [8]:
im_w, im_h = im.size

dados = dados[['class','xmin', 'ymin', 'xmax', 'ymax']]
dados['W'] = dados['xmax']-dados['xmin']
dados['H'] = dados['ymax']-dados['ymin']
dados['Norm(xMin)'] = (dados['xmin']+(dados['W']/2))/im_w
dados['Norm(yMin)'] = (dados['ymin']+(dados['H']/2))/im_h
dados['Norm(W)'] = (dados['W'])/im_w
dados['Norm(H)'] = (dados['H'])/im_h

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [168]:
dados = dados.drop(['xmin',	'ymin',	'xmax',	'ymax', 'H','W'], axis = 1)
dados.head(10)

,class,Norm(xMin),Norm(yMin),Norm(W),Norm(H)
0,0,0.492188,0.378906,0.014844,0.034091
1,5,0.506641,0.378995,0.014063,0.031783
2,0,0.593750,0.386719,0.015625,0.034801
3,4,0.567187,0.384588,0.014844,0.034801
4,6,0.552734,0.383168,0.014063,0.033381
5,9,0.521094,0.381392,0.014063,0.032670
6,8,0.538672,0.382457,0.014063,0.033381


In [121]:
len(dados)

7

In [173]:
textfile = open("/content/baixados (6).txt", "w")
for i in range(len(dados)):
  lista = dados.iloc[i].to_list()
  lista[0] = int(lista[0])
  print(lista)
  listToStr = ' '.join(map(str, lista))
  textfile.write(listToStr + "\n")

textfile.close()



    



[0, 0.4921875, 0.37890625, 0.01484375, 0.03409090909090909]
[5, 0.506640625, 0.3789950284090909, 0.0140625, 0.031782670454545456]
[0, 0.59375, 0.38671875, 0.015625, 0.03480113636363636]
[4, 0.5671875, 0.3845880681818182, 0.01484375, 0.03480113636363636]
[6, 0.552734375, 0.38316761363636365, 0.0140625, 0.033380681818181816]
[9, 0.52109375, 0.38139204545454547, 0.0140625, 0.032670454545454544]
[8, 0.538671875, 0.38245738636363635, 0.0140625, 0.033380681818181816]


In [171]:

for i in range(len(dados)):
  lista = dados.iloc[i].to_list()
  lista[0] = int(lista[0])
 
  listToStr = ' '.join(map(str, lista))
  print(listToStr)



0 0.4921875 0.37890625 0.01484375 0.03409090909090909
5 0.506640625 0.3789950284090909 0.0140625 0.031782670454545456
0 0.59375 0.38671875 0.015625 0.03480113636363636
4 0.5671875 0.3845880681818182 0.01484375 0.03480113636363636
6 0.552734375 0.38316761363636365 0.0140625 0.033380681818181816
9 0.52109375 0.38139204545454547 0.0140625 0.032670454545454544
8 0.538671875 0.38245738636363635 0.0140625 0.033380681818181816


In [111]:
np.savetxt(r'/content/baixados (6).txt', dados.values, fmt='%s')
